In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input, Conv2D
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# image crop
def crop_knee(image, crop_height, crop_width):
  h, w = image.shape[:2]
  center = (h//2, w//2)
  cropped_img = image[
      center[0] - crop_height//2 : center[0] + crop_height//2,
      center[1] - crop_width//2 : center[1] + crop_width//2
  ]
  return cropped_img

# HE CLAHE
def apply_img(image):
  clahe = cv2.createCLAHE(clipLimit=2, tileGridSize=(8, 8))
  return clahe.apply(image)

# Data Load & Preprocessing
def process_load_img(input_dir, crop_height, crop_width):
  images = []
  labels = []
  for class_folder in os.listdir(input_dir):
    class_input_path = os.path.join(input_dir, class_folder)
    class_label = int(class_folder)
    for img_file in os.listdir(class_input_path):
      img_path = os.path.join(input_dir, class_folder)
      img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
      if img is not None:
        cropped_img = crop_knee(img, crop_height, crop_width)
        clahe_img = apply_clahe(cropped_img)
        images.append(clahe_img)
        labels.append(class_label)
    return np.array(images), np.array(labels)
  
# Data Merge & Split(new dataset)
def merge_split_datas(train_dir, val_dir, crop_height, crop_width, test_size=.2):
  X_train, y_train = process_load_img(train_dir, crop_height, crop_width)
  X_val, y_val = process_load_img(val_dir, crop_height, crop_width)
  X_comb = np.concatenate((X_train, X_val), axis=0)
  y_comb = np.concatenate((y_train, y_val), axis=0)
  X_train_n, X_val_n, y_train_n, y_val_n = train_test_split(X_comb, y_comb, test_size=test_size, random_state=42, stratify=y_comb)
  return X_train_n, X_val_n, y_train_n, y_val_n

In [3]:
input_shape = (crop_height, crop_width)
class_num = 5

def create_model(model_name='Resnet50', optimizer='adam', dropout_rate=.5, fc_units=512, class_num=class_num):
  input_layer = Input(shape=input_shape)
  conv_layer = Conv2D(3, (3, 3), padding='same',)(input_layer)
  if model_name == 'ResNet50':
    base = ResNet50(weights='imagenet', include_top=False, input_shape=(crop_height, crop_width,3))
  else:
    base = DenseNet121(weights='imagenet', include_top=False, input_shape=(crop_height, crop_width,3))
  base_out = base(conv_layer)
  flatten = Flatten()(base_out)
  fc = Dense(fc_units, activation='relu',)(flatten)
  dropout = Dropout(dropout_rate)(fc)
  output = Dense(class_num, activation='softmax')(dropout)
  model = Model(inputs=input_layer, outputs=output)
  model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
  return model
model = KerasClassifier(build_fn=create_model, verbose=0)
param_grid = {
    'model_name':['ResNet50', 'DenseNet', ],
    'batch_size':[16, 32],
    'epochs':[30, 50, 80],
    'optimizer':['adam', 'rmsprop',],
    'dropout_rate':[0.2, 0.4, 0.5],
    'fc_units':[512, 1024]
}
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_res = grid.fit(X_train, y_train)
means = grid_res.cv_results_['mean_test_score']
std = grid_res.cv_results_['std_test_score']
params = grid_res.cv_results_['params']
best_model = grid_res.best_estimator_.model
test_loss, test_acc = best_model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_acc}")

NameError: name 'crop_height' is not defined